# Breast Cancer Detection using Machine Learning

##  enviroment and global variables 

In [7]:
! pip install tensorflow-gpu

     |████████████████████████████████| 421.8MB 22kB/s 
     |████████████████████████████████| 450kB 43.8MB/s 
     |████████████████████████████████| 3.9MB 59.4MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.1.1 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.1.0 which is incompatible.
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0


In [1]:
import os
import glob
import numpy as np
import pandas as pd

from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import shuffle
from sklearn.utils.class_weight import compute_class_weight


import matplotlib.pyplot as plt
import cv2


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

print(tf.__version__)
print(tf.test.gpu_device_name())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.1.0
/device:GPU:0
Num GPUs Available:  1


In [4]:
!ls

sample_data


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
TRAIN_CSV = "./train/feats.csv"
TRAIN_IMG_DIR = "./train/images"

TEST_CSV = "./test/feats.csv"
TEST_IMG_DIR = "./test/images"

CLEAN_DATA_FILE = "./cleaned_data.csv"

MODEL_PATH = "./"
RESULT_FILE = "./submission.csv"

IMG_HEIGHT = 320
IMG_WIDTH = 320

## 1. data anaylysis

In [6]:
train_df = pd.read_csv(TRAIN_CSV)
train_df.head()

,id,age,HER2,P53,molecular_subtype
0,8a1af6f74cafc720511888998f2f361767c77965a07464...,48,3,True,3
1,b18adad8c39bb458f208581fe40e9be7b04f2b49be04b0...,54,3,False,2
2,8b9af020d0e621168fd0f3913f2cd74e8bb5ea85f61ee0...,55,2,True,3
3,1a0e54f07794fbb95a0fd7f0b4ef6f79ef1a9c83dbaa05...,66,0,False,4
4,2805706a656b013743d84357d5ffa44536e19d18f0509f...,31,0,False,1


In [7]:
print(train_df.shape)
train_df.groupby('molecular_subtype')['id'].nunique()

(200, 5)


molecular_subtype
1    58
2    98
3    33
4    11
Name: id, dtype: int64

In [8]:
test_df = pd.read_csv(TEST_CSV)
test_df.head()

,id,age,HER2,P53
0,d6b47f0c2ccbbd7923e37dc434ab25445df6bc060d5338...,57,0,False
1,2e9d400ee8450c30d19c43a20e9da770774d3169da4574...,36,0,False
2,72f92da6ca6d521221baee367b5c76fcbfe61edc00c0bc...,81,0,False
3,ca7e160144dd6f459e2b18c48a33d4a53d09de5d7c2557...,82,0,False
4,d65889918d2b9073e7601c2ba67c3a9d26f60612f14e93...,59,2,False


## data preprocessing

### each id may have more than one image. Create 1 to 1 mapping between image path and label. 

In [9]:
f = open(CLEAN_DATA_FILE, "w")
for idx, row in train_df.iterrows():
    image_path_list = glob.glob(os.path.join(TRAIN_IMG_DIR, row["id"], "*.*"))
    for image_path in image_path_list:
        f.write(f'{image_path}, {row["molecular_subtype"]}\n')

f.close()
print(f"cleaned mapping stored in {CLEAN_DATA_FILE}.")

cleaned mapping stored in ./cleaned_data.csv.


In [10]:
print(dir(tf.keras.applications))
'''
base_model = tf.keras.applications.ResNet50(input_shape=(480, 480, 3),
                                               include_top=False,
                                               weights='imagenet')
                                               '''

['DenseNet121', 'DenseNet169', 'DenseNet201', 'InceptionResNetV2', 'InceptionV3', 'MobileNet', 'MobileNetV2', 'NASNetLarge', 'NASNetMobile', 'ResNet101', 'ResNet101V2', 'ResNet152', 'ResNet152V2', 'ResNet50', 'ResNet50V2', 'VGG16', 'VGG19', 'Xception', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_sys', 'densenet', 'imagenet_utils', 'inception_resnet_v2', 'inception_v3', 'mobilenet', 'mobilenet_v2', 'nasnet', 'resnet', 'resnet50', 'resnet_v2', 'vgg16', 'vgg19', 'xception']


"\nbase_model = tf.keras.applications.ResNet50(input_shape=(480, 480, 3),\n                                               include_top=False,\n                                               weights='imagenet')\n                                               "

### process training image path and convert labels to one-hot encoding

In [11]:
train_image_path = []
train_raw_label = []
with open(CLEAN_DATA_FILE, "r") as f:
    lines = f.readlines()
    for line in lines:
        image_path, label = line.split(",")
        label = label.strip() 
        train_image_path.append(image_path)
        train_raw_label.append(label)

        

#print(list(train_raw_label))
from collections import Counter
c = Counter(list(train_raw_label))
print(c)        

train_image_path = np.asarray(train_image_path)
train_raw_label = np.asarray(train_raw_label).reshape(-1, 1)
train_image_path, train_raw_label = shuffle(train_image_path, train_raw_label)

encoder = OneHotEncoder(sparse=False)
train_onehot_label = encoder.fit_transform(train_raw_label)

#[print(i, j, k) for i, j, k in zip(train_image_path, train_onehot_label, train_raw_label)]

print(f"{train_image_path.shape}, {train_onehot_label.shape}")




Counter({'2': 472, '1': 267, '3': 136, '4': 50})
(925,), (925, 4)


### image preprocessing function

In [13]:
def preprocess_image(image_path):
    
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH))
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    
    image = cv2.normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

    
    image = np.asarray(image)
    #print(image_path, np.max(image), np.min(image), np.mean(image))
    #image = image /255.0
    #image -= 0.5
    #print(image_path, np.max(image), np.min(image), np.mean(image))
    
    #print(image.shape)
    return image

train_X = []
for image_path in train_image_path:
    train_X.append(preprocess_image(image_path))

train_X = np.asarray(train_X)
#train_X = np.expand_dims(train_X)


train_Y = train_onehot_label

print(train_X.shape, train_Y.shape)

#[print(j) for i, j in zip(train_X, train_Y)]


(925, 320, 320, 3) (925, 4)


## build machine learning model 

In [0]:
def build_model():
    model = Sequential()
    model.add(Conv2D(16, 3, padding='valid', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)))
    model.add(MaxPooling2D())

    model.add(Conv2D(32, 3, padding='valid', activation='relu'))
    model.add(MaxPooling2D())
    model.add(Conv2D(64, 3, padding='valid', activation='relu'))
    model.add(MaxPooling2D())
    model.add(Conv2D(128, 3, padding='valid', activation='relu'))
    model.add(MaxPooling2D())

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(32, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(4, activation='softmax'))
    
    
    model.summary()
    return model



In [0]:
def build_transfer_learning_model(base_model):
    
    base_model.trainable = False
    
    model = Sequential()
    model.add(base_model)
    model.add(Flatten())
    #model.add(Dense(256, activation = "relu"))
    model.add(Dense(128, activation = "relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(64, activation = "relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(32, activation = "relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(4, activation='softmax'))
    
    model.summary()
    return model

In [0]:
datagen = ImageDataGenerator(
    featurewise_center=False,
    featurewise_std_normalization=False,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

datagen.fit(train_X)


In [17]:
print(train_X[30])

[[[0.09803922 0.09803922 0.09803922]
  [0.09803922 0.09803922 0.09803922]
  [0.09803922 0.09803922 0.09803922]
  ...
  [0.09803922 0.09803922 0.09803922]
  [0.09803922 0.09803922 0.09803922]
  [0.09803922 0.09803922 0.09803922]]

 [[0.00784314 0.00784314 0.00784314]
  [0.00784314 0.00784314 0.00784314]
  [0.00784314 0.00784314 0.00784314]
  ...
  [0.00784314 0.00784314 0.00784314]
  [0.00784314 0.00784314 0.00784314]
  [0.00784314 0.00784314 0.00784314]]

 [[0.00392157 0.00392157 0.00392157]
  [0.00392157 0.00392157 0.00392157]
  [0.00392157 0.00392157 0.00392157]
  ...
  [0.00392157 0.00392157 0.00392157]
  [0.00392157 0.00392157 0.00392157]
  [0.00392157 0.00392157 0.00392157]]

 ...

 [[0.         0.         0.        ]
  [0.         0.         0.        ]
  [0.         0.         0.        ]
  ...
  [0.         0.         0.        ]
  [0.         0.         0.        ]
  [0.         0.         0.        ]]

 [[0.         0.         0.        ]
  [0.         0.         0.        ]


### simple CNN

In [21]:
model =  build_model() #build_transfer_learning_model() # #

optimizer = Adam(
    learning_rate=0.001, epsilon=1e-07, amsgrad=False,)

model.compile(optimizer=optimizer,
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


y_integers = np.argmax(train_Y, axis=1)
class_weights = compute_class_weight('balanced', np.unique(y_integers), y_integers)
d_class_weights = dict(enumerate(class_weights))

print("class_weight: ", d_class_weights)


history = model.fit(datagen.flow(train_X, train_Y, batch_size = 32),  epochs=100, shuffle = True, class_weight = d_class_weights)
                   

model.save(os.path.join(MODEL_PATH, "simple_cnn_model.h5"))
print("model saved.")

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 318, 318, 16)      448       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 159, 159, 16)      0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 157, 157, 32)      4640      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 78, 78, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 76, 76, 64)        18496     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 38, 38, 64)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 36, 36, 128)      

### transfer Learning

In [31]:
base_model = tf.keras.applications.DenseNet121(input_shape=(IMG_HEIGHT, IMG_WIDTH ,3), include_top=False, weights='imagenet')

transfer_model = build_transfer_learning_model(base_model) # #build_model() #

optimizer = Adam(
    learning_rate=0.001, epsilon=1e-07, amsgrad=False,)

transfer_model.compile(optimizer=optimizer,
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


y_integers = np.argmax(train_Y, axis=1)
class_weights = compute_class_weight('balanced', np.unique(y_integers), y_integers)
d_class_weights = dict(enumerate(class_weights))

print("class_weight: ", d_class_weights)


history = transfer_model.fit(datagen.flow(train_X, train_Y, batch_size = 32),  epochs=100, shuffle = True, class_weight = d_class_weights)
                   

transfer_model.save(os.path.join(MODEL_PATH, "transfer_learning_model.h5"))

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Model)          (None, 10, 10, 1024)      7037504   
_________________________________________________________________
flatten_5 (Flatten)          (None, 102400)            0         
_________________________________________________________________
dense_17 (Dense)             (None, 128)               13107328  
_________________________________________________________________
batch_normalization_14 (Batc (None, 128)               512       
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 64)                8256      
_________________________________________________________________
batch_normalization_15 (Batc (None, 64)               

In [0]:
def predict(model, image_path):
    image = preprocess_image(image_path)
    prediction =  model.predict(np.asarray([image]))
    print(prediction)
    result = np.argmax(prediction) + 1
    return result




## create final result

In [32]:
def create_result(model, csv_path, image_dir):
    test_df = pd.read_csv(csv_path)
    test_image_id = list(test_df["id"])
    #print(test_image_id)
    f = open(RESULT_FILE, "w")
    for idx in test_image_id:
        test_image_file_path = glob.glob(os.path.join(image_dir, idx, "*.*"))
        #print(test_image_file_path)
        result = []
        
        for p in test_image_file_path:
            result.append(predict(model, p))
        
        #print(result)
        result = np.asarray(result)
        if result.shape[0] != 0:
            counts = np.bincount(result)
            final = np.argmax(counts)
            print(f"{idx}, {result}, {final}")
        else:
            print(f"{idx} NO result")
            final = 1
        
        f.write(f"{idx},{final}\n")
    
    f.close()
    print("done")

    
create_result(transfer_model, TEST_CSV, TEST_IMG_DIR)
    

[[0.0013046  0.04962983 0.9479104  0.00115516]]
[[6.9129653e-04 4.3947753e-02 9.5423824e-01 1.1227359e-03]]
[[6.7708589e-04 3.3207841e-02 9.6534216e-01 7.7298982e-04]]
[[0.00102927 0.11817577 0.8783619  0.00243316]]
[[8.2536229e-05 1.4108918e-02 9.8539346e-01 4.1508293e-04]]
[[0.00405826 0.1415137  0.852174   0.00225406]]
d6b47f0c2ccbbd7923e37dc434ab25445df6bc060d53388d0e16237b669d19d2, [3 3 3 3 3 3], 3
[[4.0995923e-04 8.5637206e-03 7.1201593e-01 2.7901036e-01]]
[[0.0007107  0.01465465 0.62043244 0.36420217]]
2e9d400ee8450c30d19c43a20e9da770774d3169da457490fd99d9dfb837419c, [3 3], 3
[[0.00137807 0.12481585 0.8671824  0.00662377]]
[[0.00257825 0.17130096 0.8224098  0.00371093]]
[[0.00141788 0.09268376 0.89660895 0.00928939]]
[[0.00884756 0.28516555 0.701154   0.00483279]]
72f92da6ca6d521221baee367b5c76fcbfe61edc00c0bcce72fae35dadcd6185, [3 3 3 3], 3
[[6.9818768e-04 8.2801608e-04 2.9347183e-02 9.6912652e-01]]
[[0.01174368 0.09832747 0.8865903  0.00333848]]
[[3.2603260e-04 5.7946010e-03 8